In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Location/raw_location_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['Unnamed: 0.1', 'Unnamed: 0_x', 'uid', 'date', 'number_places_morning',
       'number_places_afternoon', 'number_places_evening',
       'number_places_night', 'variance_time_spent_morning',
       'variance_time_spent_afternoon', 'variance_time_spent_evening',
       'variance_time_spent_night', 'time_spent_morning_firstplace',
       'time_spent_afternoon_firstplace', 'time_spent_evening_firstplace',
       'time_spent_night_firstplace', 'time_spent_morning_secondplace',
       'time_spent_afternoon_secondplace', 'time_spent_evening_secondplace',
       'time_spent_night_secondplace', 'time_spent_morning_thirdplace',
       'time_spent_afternoon_thirdplace', 'time_spent_evening_thirdplace',
       'time_spent_night_thirdplace', 'Unnamed: 0_y',
       'time_spent_morning_1stplace', 'time_spent_afternoon_1stplace',
       'time_spent_evening_1stplace', 'time_spent_night_1stplace',
       'time_spent_morning_2ndplace', 'time_spent_afternoon_2ndplace',
       'time_spent_evening_

In [4]:
df.isnull().sum()
del df['Unnamed: 0.1']
del df['Unnamed: 0_x']

In [5]:
df = df.dropna()
print(len(df))

1203


In [6]:
df.columns

Index(['uid', 'date', 'number_places_morning', 'number_places_afternoon',
       'number_places_evening', 'number_places_night',
       'variance_time_spent_morning', 'variance_time_spent_afternoon',
       'variance_time_spent_evening', 'variance_time_spent_night',
       'time_spent_morning_firstplace', 'time_spent_afternoon_firstplace',
       'time_spent_evening_firstplace', 'time_spent_night_firstplace',
       'time_spent_morning_secondplace', 'time_spent_afternoon_secondplace',
       'time_spent_evening_secondplace', 'time_spent_night_secondplace',
       'time_spent_morning_thirdplace', 'time_spent_afternoon_thirdplace',
       'time_spent_evening_thirdplace', 'time_spent_night_thirdplace',
       'Unnamed: 0_y', 'time_spent_morning_1stplace',
       'time_spent_afternoon_1stplace', 'time_spent_evening_1stplace',
       'time_spent_night_1stplace', 'time_spent_morning_2ndplace',
       'time_spent_afternoon_2ndplace', 'time_spent_evening_2ndplace',
       'time_spent_night_2nd

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    482
low stress       363
high stress      358
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
len(binary_lh_data['uid'].unique())

46

In [10]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

logo = LeaveOneGroupOut()

In [11]:
print(len(groups))

721


In [12]:
best_thresholds = []
balanced_accs = []
auc_scores = []

In [13]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
num_splits = len(np.unique(groups))
print(num_splits)

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print('X and y train shapes: ')
        print(X_train.shape)
        print(y_train.shape)

        train_class_counts = np.bincount(y_train)
        print(f"Train set - Class 0 (Low Stress): {train_class_counts[0]}, "
              f"Class 1 (High Stress): {train_class_counts[1] if len(train_class_counts) > 1 else 0}")

        test_class_counts = np.bincount(y_test)
        print(f"Test set - Class 0 (Low Stress): {test_class_counts[0]}, "
              f"Class 1 (High Stress): {test_class_counts[1] if len(test_class_counts) > 1 else 0}")

        print(f"Test set size: {len(y_test)}")
        print(f"Train set shape: {X_train.shape}")
        print(f"Test set shape: {X_test.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(X_train_normalized.shape)
        print(X_test_normalized.shape)

        # Define the FCN model
        model_fcn_binary_lh = tf.keras.models.Sequential([
            tf.keras.layers.Dense(57, activation='relu', input_dim=X_train_normalized.shape[1]),
            tf.keras.layers.Dropout(0.35),
            tf.keras.layers.Dense(35, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(3, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
        ])
        print('Model Defined')

        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model_fcn_binary_lh.compile(
            optimizer=custom_optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']  
        )

        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(enumerate(class_weights))

        print('Class Weight: ', class_weight_dict)

        model_fcn_binary_lh.fit(
            X_train_normalized, y_train,
            epochs=100,
            batch_size=32,
            class_weight=class_weight_dict,
            verbose=1  
        )

        thresholds = np.arange(0.01, 1.0, 0.01)
        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        best_threshold = 0.0
        best_metric = 0.0

        for threshold in thresholds:
            y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
            metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)

            if metric_value > best_metric:
                best_metric = metric_value
                best_threshold = threshold

        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)  
            if auc_score < 0.5:
                print(f"<0.5 AUC Score: {auc_score}")
                print("----------------------------------------")
            else:
                print(f"AUC Score: {auc_score}")

        else:
            auc_scores.append(None)  
            print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc) 
        best_thresholds.append(best_threshold)  
        print(f"Balanced Accuracy: {balanced_acc}")

        pbar.update(1)

46


LOSO CV Progress:   0%|                                    | 0/46 [00:00<?]   0%

X and y train shapes: 
(696, 81)
(696,)
Train set - Class 0 (Low Stress): 351, Class 1 (High Stress): 345
Test set - Class 0 (Low Stress): 12, Class 1 (High Stress): 13
Test set size: 25
Train set shape: (696, 81)
Test set shape: (25, 81)
(696, 81)
(25, 81)
Model Defined


Class Weight:  {0: 0.9914529914529915, 1: 1.008695652173913}
Epoch 1/100


2024-10-05 10:46:10.532734: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


...

Skipping AUC computation for this fold as y_test contains only one class: [0]
Balanced Accuracy: 1.0
X and y train shapes: 
(705, 81)
(705,)
Train set - Class 0 (Low Stress): 352, Class 1 (High Stress): 353
Test set - Class 0 (Low Stress): 11, Class 1 (High Stress): 5
Test set size: 16
Train set shape: (705, 81)
Test set shape: (16, 81)
(705, 81)
(16, 81)


Model Defined
Class Weight:  {0: 1.0014204545454546, 1: 0.9985835694050992}
Epoch 1/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7373 - accuracy: 0.4610
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6965 - accuracy: 0.5262
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6835 - accuracy: 0.5277
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7002 - accuracy: 0.5262
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6885 - accuracy: 0.5021
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6903 - accuracy: 0.5149
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6846 - accuracy: 0.5206
Epoch 8/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6750 - accuracy: 0.5518
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6768 - accuracy: 0.5348
Epoch 10/100
23/23 [=============

LOSO CV Progress:  63%|██████████████████▉            | 29/46 [01:44<00:59]  63%


----------------------------------------
Balanced Accuracy: 0.6363636363636364


X and y train shapes: 
(709, 81)
(709,)
Train set - Class 0 (Low Stress): 361, Class 1 (High Stress): 348
Test set - Class 0 (Low Stress): 2, Class 1 (High Stress): 10
Test set size: 12
Train set shape: (709, 81)
Test set shape: (12, 81)
(709, 81)
(12, 81)
Model Defined
Class Weight:  {0: 0.981994459833795, 1: 1.0186781609195403}
Epoch 1/100
23/23 [==============================] - 0s 2ms/step - loss: 0.7414 - accuracy: 0.4937
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6919 - accuracy: 0.5190
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 0.7019 - accuracy: 0.5233
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6813 - accuracy: 0.5416
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6806 - accuracy: 0.5599
Epoch 6/100
 1/23 [>.............................] - ETA: 0s - loss: 0.7098 - accuracy: 0.5625

In [ ]:
from sklearn.linear_model import LogisticRegression

num_splits = len(np.unique(groups))
print(num_splits)

auc_scores = []
balanced_accs = []
best_thresholds = []

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print('X and y train shapes: ')
        print(X_train.shape)
        print(y_train.shape)

        # train_class_counts = np.bincount(y_train)
        # print(f"Train set - Class 0 (Low Stress): {train_class_counts[0]}, Class 1 (High Stress): {train_class_counts[1] if len(train_class_counts) > 1 else 0}")

        # test_class_counts = np.bincount(y_test)
        # print(f"Test set - Class 0 (Low Stress): {test_class_counts[0]}, Class 1 (High Stress): {test_class_counts[1] if len(train_class_counts) > 1 else 0}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        model_logistic = LogisticRegression(class_weight='balanced', max_iter=1000)

        model_logistic.fit(X_train_normalized, y_train)
        y_test_pred_proba = model_logistic.predict_proba(X_test_normalized)[:, 1]

        thresholds = np.arange(0.01, 1.0, 0.01)
        best_threshold = 0.0
        best_metric = 0.0

        for threshold in thresholds:
            y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
            metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)

            if metric_value > best_metric:
                best_metric = metric_value
                best_threshold = threshold

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc)
        best_thresholds.append(best_threshold)
        print(f"Balanced Accuracy: {balanced_acc}")

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)
            print(f"AUC Score: {auc_score}")
        else:
            auc_scores.append(None)
            print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        pbar.update(1)

In [ ]:
print('LR: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

In [ ]:
print('MLP: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")